In [30]:
import pickle
import numpy as np

from ner_utils.Sentence import Sentence
from ner_utils.Data     import Data

import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, Dense, LSTM, Embedding, Concatenate, Reshape, Permute, Lambda, Bidirectional

from tensorflow_addons.layers import CRF

### Reading Dataset

In [2]:
data = Data('data/ner_datasetreference.csv', encoding='latin1')
sentence = Sentence(data.dataset)

## Data Features

In [3]:
# getting char voacb
# with open('unique_chars.pkl', 'wb') as f:
#     pickle.dump(sentence.get_unique_chars(), f)

# loading char vocab   
with open('unique_chars.pkl', 'rb') as f:
    chars = pickle.load(f)

In [4]:
# Word info to pass to the model
max_word_len, min_word_len, avg_word_len, std_word_len = sentence.get_word_info()
avg_word_len, std_word_len

(4.773359082564433, 2.8246976648249014)

In [5]:
# Sentence info to pass to the model
max_sentence_len, min_sentence_len, avg_sentence_len, std_sentence_len = sentence.get_sentence_info()
max_sentence_len, min_sentence_len, avg_sentence_len, std_sentence_len

(104, 1, 21.863987989741236, 7.963596820721575)

## Tokens

In [6]:
word_list, sentence_list = sentence.get_words_and_sentences()
word_sequences, sentence_sequences = data.get_tokenized_sequences(word_list, sentence_list, max_word_len, max_sentence_len)

In [7]:
char_to_idx, word_to_idx, idx_to_char, idx_to_word = data.get_tokens(word_list, sentence_list)

### Char Embeddings

In [8]:
#Word Level Hyper Parameters
model_word_len     = int(np.ceil(avg_word_len + 2*std_word_len))
char_embedding_dim = 35
char_vocab_size    = len(char_to_idx) + 1 #add 1 for padding token
window_size        = 3
conv_filters       = 30 


#Sentence Level Hyper Parameters
model_sentence_len = int(np.ceil(avg_sentence_len + 2*std_sentence_len))
word_embedding_dim = 100
word_vocab_size    = len(word_to_idx) + 1 #add 1 for padding token

#Other Hyper Pararmeters
dropout_rate    = 0.5
n_tags          = 10 

model_word_len, model_sentence_len, word_vocab_size

(11, 38, 31819)

In [9]:
def generate_embedding_matrix_from_file(embedding_filename, vocab_size, embedding_dim, key_to_idx):
    
    with open(embedding_filename, 'r') as f:
        # char_embeddings = pickle.load(f)
    
        embedding_matrix = np.zeros((vocab_size, embedding_dim))
        for line in f.readlines():            
            embedding_matrix[char_to_idx[line.split[0]]] = np.array(line.split[1:], dtype=np.float32)
        
    return embedding_matrix  

In [10]:
char_embedding_matrix = get_char_embedding_matrix('char_embeddings_with_features.pkl', char_vocab_size, char_embedding_dim, char_to_idx)


In [11]:
#word_embedding_martrix = get_char_embedding_matrix()

### Model Input and Labels

In [100]:
# word and sentence inputs
word_inputs     = word_list
sentence_inputs = sentence_list

# target labels
y = sentence.tags

### Defining the Model

In [13]:
#Word Part of Model
char_inputs = Input(shape=(model_sentence_len, model_word_len))
x           = Lambda(lambda x: tf.reshape(x, (model_sentence_len, model_word_len)))(char_inputs)
x           = Embedding(char_vocab_size,
                            char_embedding_dim,
                            weights=[char_embedding_matrix],
                            input_length=max_word_len,
                            trainable=True)(x)
x           = Permute((2, 1), input_shape=(11, 35))(x)
x           = Dropout(dropout_rate)(x)
x           = Reshape((35, 11, 1))(x)
x           = Conv2D(conv_filters, input_shape=(38, 35, 11, 1), kernel_size=(1, window_size), padding='same')(x)
x           = MaxPooling2D(pool_size=(1, model_word_len))(x)
x           = Dense(1, input_shape=(35, 1, 30))(x)
x           = Reshape((35, ))(x)
x           = Lambda(lambda x: tf.expand_dims(x, axis=0))(x)


#Sentence Part of Model
word_inputs = Input(shape=(model_sentence_len, ))
y           = Embedding(word_vocab_size,
                            word_embedding_dim,
                            #weights=[word_embedding_matrix],
                            input_length=max_sentence_len,
                            trainable=False)(word_inputs)


#Joining the two parts
z = Concatenate(axis=2)([y, x])
z = Dropout(dropout_rate)(z)
z = Bidirectional(LSTM(model_sentence_len, return_sequences=True, recurrent_dropout=dropout_rate))(z)
z = Dropout(dropout_rate)(z)
z = CRF(n_tags)(z)
# outputs, _,  _, _ = z
outputs = z


C:\Users\laraibshahzil\AppData\Roaming\Python\Python37\site-packages\tensorflow\python\autograph\impl\api.py:376: UserWarning: CRF decoding models have serialization issues in TF >=2.5 . Please see isse #2476
  return py_builtins.overload_of(f)(*args)


In [14]:
tf.random.set_seed(10)
model = Model(inputs=[char_inputs, word_inputs], outputs=outputs)

ValueError: Output tensors of a Functional model must be the output of a TensorFlow `Layer` (thus holding past layer metadata). Found: <tensorflow_addons.layers.crf.CRF object at 0x00000191E30F4B80>

In [15]:
model.summary()

Model: "model_6"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 38, 11)]     0                                            
__________________________________________________________________________________________________
lambda (Lambda)                 (38, 11)             0           input_1[0][0]                    
__________________________________________________________________________________________________
embedding (Embedding)           (38, 11, 35)         3500        lambda[0][0]                     
__________________________________________________________________________________________________
permute (Permute)               (38, 35, 11)         0           embedding[0][0]                  
____________________________________________________________________________________________

In [18]:
from tensorflow.keras.optimizers import Adam, RMSprop

model.compile(optimizer="Adam", loss="mse", metrics=["mae"])


In [28]:
history = model.fit([word_sequences, sentence_sequences], y)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [21]:
np.random.seed(20)
char_inputs_1 = np.random.randint(10, size=(1, 38, 11))
word_inputs_1 = np.random.randint(10, size=(1, 38))

print(f"Char Inputs Shape: {char_inputs.shape}\nWord Inputs Shape: {word_inputs.shape}")



Char Inputs Shape: (None, 38, 11)
Word Inputs Shape: (None, 38)


In [ ]:
output = model.predict([char_inputs_1, word_inputs_1], batch_size=1)
decoded_sequence, potentials, sequence_length, chain_kernel = output
# output

In [102]:
decoded_sequence

In [ ]:
potentials[0][0]

In [ ]:
model

In [36]:
m = Sequential([
        Input(shape=(1,)),
        Dense(3, activation="relu"),
        Dense(1),
    ])
m.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_8 (Dense)              (None, 3)                 6         
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 4         
Total params: 10
Trainable params: 10
Non-trainable params: 0
_________________________________________________________________


In [52]:
from tensorflow.keras.optimizers import Adam, RMSprop

m.compile(optimizer=Adam(0.1), loss="mse")


In [53]:
x = [i for i in range(20)]
y = [i*2 for i in range(20)]

In [55]:
m.fit(x, y, epochs=30, verbose=2)

Epoch 1/30
1/1 - 0s - loss: 44.3836
Epoch 2/30
1/1 - 0s - loss: 37.7376
Epoch 3/30
1/1 - 0s - loss: 26.5999
Epoch 4/30
1/1 - 0s - loss: 15.1773
Epoch 5/30
1/1 - 0s - loss: 6.4348
Epoch 6/30
1/1 - 0s - loss: 1.7343
Epoch 7/30
1/1 - 0s - loss: 0.9961
Epoch 8/30
1/1 - 0s - loss: 3.1555
Epoch 9/30
1/1 - 0s - loss: 6.7254
Epoch 10/30
1/1 - 0s - loss: 10.2882
Epoch 11/30
1/1 - 0s - loss: 12.8075
Epoch 12/30
1/1 - 0s - loss: 13.7434
Epoch 13/30
1/1 - 0s - loss: 13.0285
Epoch 14/30
1/1 - 0s - loss: 10.9691
Epoch 15/30
1/1 - 0s - loss: 8.1168
Epoch 16/30
1/1 - 0s - loss: 5.1309
Epoch 17/30
1/1 - 0s - loss: 2.6381
Epoch 18/30
1/1 - 0s - loss: 1.0991
Epoch 19/30
1/1 - 0s - loss: 0.7013
Epoch 20/30
1/1 - 0s - loss: 1.3111
Epoch 21/30
1/1 - 0s - loss: 2.5171
Epoch 22/30
1/1 - 0s - loss: 3.7710
Epoch 23/30
1/1 - 0s - loss: 4.5801
Epoch 24/30
1/1 - 0s - loss: 4.6738
Epoch 25/30
1/1 - 0s - loss: 4.0703
Epoch 26/30
1/1 - 0s - loss: 3.0244
Epoch 27/30
1/1 - 0s - loss: 1.8978
Epoch 28/30
1/1 - 0s - loss:

In [56]:
m.predict([])

array([[14.180725]], dtype=float32)